In [18]:
import pandas as pd 
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected" # or "vscode"

from IPython.display import display 

In [19]:
eua_spot = pd.read_csv('EUA_Spot_Daily_T3PA_2023-2024.csv')
eua_price_fig = px.line(eua_spot, x = 'Date', y = 'Spot_Price_EUR', title = 'Eua spot price 2024-2024')
display(eua_price_fig)

In [20]:
eua_futures_spot = pd.read_csv('European Union Allowance (EUA) Yearly Futures Historical Data UK.csv')
eua_futures_price_fig = px.line(eua_futures_spot, x = 'Date', y = 'Price', title = 'Eua Futures price 2023-2024')
display(eua_futures_price_fig)



### Merge Data Frames, Spot and Future


In [21]:
merged_df = pd.merge(eua_spot, eua_futures_spot, on = 'Date', how = 'inner', suffixes = ('spot','futures'))
merged_df 

,Date,Spot_Price_EUR,Total_Volume_tCO2,Avg_Cover_Ratio,Price,Open,High,Low,Vol.,Change %
0,09/01/2023,75.27,2409000,2.29,81.00,77.64,81.88,77.01,16.09K,4.35%
1,10/01/2023,78.61,2409000,1.97,80.80,81.78,82.51,80.07,11.72K,-0.25%
2,12/01/2023,76.80,2409000,2.20,80.17,78.77,80.86,78.50,11.81K,1.44%
3,13/01/2023,76.42,1939500,2.11,79.63,80.50,80.56,78.21,13.86K,-0.67%
4,16/01/2023,75.05,2409000,1.86,77.52,79.72,79.75,77.37,16.42K,-2.65%
...,...,...,...,...,...,...,...,...,...,...
375,09/12/2024,67.70,3287500,1.40,66.37,68.55,68.70,65.93,47.07K,-2.84%
376,10/12/2024,67.00,3287500,1.67,68.18,66.35,68.72,66.15,42.96K,2.73%
377,12/12/2024,67.95,3287500,1.63,66.10,68.53,68.84,65.54,47.62K,-3.69%
378,13/12/2024,66.07,1890500,1.82,64.43,65.81,66.61,64.15,20.48K,-2.53%


In [22]:

# 1. Prepare Data
merged_df['Date'] = pd.to_datetime(merged_df['Date'], dayfirst=True, errors='coerce')

# 2. Create Figure (Plotly Express handles multiple lines and legends automatically)
fig = px.line(merged_df, x='Date', y=['Spot_Price_EUR', 'Price'], 
              title='EUA Futures and Spot Price 2023-2024',
              labels={'value': 'Price (€/tCO2)', 'variable': 'Type'},
              color_discrete_map={'Spot_Price_EUR': '#636efa', 'Price': '#EF553B'})

# 3. Quick Layout Tweaks
fig.update_layout(hovermode='x unified', width=1000, height=600,
                  legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

# 4. Print Summary & Display
for col in ['Spot_Price_EUR', 'Price']:
    print(f"{col} Range: {merged_df[col].min():.2f} to {merged_df[col].max():.2f}")

fig.show()

Spot_Price_EUR Range: 49.50 to 96.33
Price Range: 52.51 to 100.29


##### Find the ratio between the two assest using linear regression


In [26]:
lingress_fig = px.scatter(merged_df, x = 'Price', y = 'Spot_Price_EUR', title = 'linear regression between spot and fuitures EUA', trendline = 'ols')
display(lingress_fig)

In [30]:
from statsmodels.regression import linear_model

model = linear_model.OLS(merged_df['Price'], merged_df['Spot_Price_EUR']) 
result = model.fit()

ratio = result.params['Spot_Price_EUR']
print('Ratio:', ratio)

Ratio: 1.0220991810798643


### Adjust the price and compute the spread

In [38]:
merged_df['Spot_Price_adjusted'] = merged_df['Price'] * ratio 

# Add spread to the dataframe
merged_df['spread'] = merged_df['Price'] - merged_df['Spot_Price_adjusted']

# Plot adjusted prices (multiple columns - use list)
price_adj_fig = px.line(merged_df, x = 'Date', y = ['Price', 'Spot_Price_adjusted'],
                        title='Price vs Adjusted Spot Price',
                        labels={'value': 'Price (€/tCO2)', 'variable': 'Type'})
display(price_adj_fig)

# Plot spread (single column - use string, not list)
spread_fig = px.line(merged_df, x = 'Date', y = 'spread',
                     title='Spread: Price - Adjusted Spot Price',
                     labels={'spread': 'Spread (€/tCO2)'})
display(spread_fig)